In [2]:
import numpy as np
import pandas as pd


In [4]:
train = pd.read_csv('/mnt/c/users/ventu/Python/Kaggle/House Prices/train.csv')
test = pd.read_csv('//mnt/c/users/ventu/Python/Kaggle/House Prices/test.csv')
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


First I want to check for how missing values and evaluate based of percentage missing whether I want to drop the column or not.

1) Alley, FireplaceQu, PoolQC, Fence, MiscFeature are missing 50% or more entries

    a) MiscFeature are very niche items (ie Tennis Court) which would skew our model so I will remove that column

    b) 

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


In [5]:
xp_train = train.copy()
xp_train.drop
xp_train.GarageYrBlt.fillna(xp_train.YearBuilt, inplace=True)
s = (xp_train.dtypes == 'object')
obj_col = list(s[s].index)

In [6]:
X = xp_train.drop(['SalePrice', 'MiscFeature', 'Alley', 'PoolQC', 'Fence' , 'FireplaceQu'], axis=1)
y = xp_train.SalePrice

numerical_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]

categorical_cols = [col for col in X.columns if X[col].dtype == "object"]

train_X, val_X, train_y, val_y = train_test_split(X, y, train_size=0.8, test_size=0.2)

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

numerical_transformer = SimpleImputer(strategy='median')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

model = RandomForestRegressor(n_estimators=1000, random_state=0)
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])
clf.fit(train_X, train_y)
preds = clf.predict(val_X)

In [8]:
print(mean_absolute_error(val_y, preds))
print(clf.score(val_X,val_y))

17841.74954452055
0.8538167461181141


In [9]:
from xgboost import XGBRegressor

model_2 = XGBRegressor(n_estimators=1000)
model_2_pl = Pipeline(steps=[('preprocessor', preprocessor),
                            ('model_2',model_2)])

model_2_pl.fit(train_X, train_y)
prediction = model_2_pl.predict(val_X)

print(mean_absolute_error(val_y, prediction))
print(model_2_pl.score(val_X,val_y))


17524.218121254282
0.8867700668980507
